# 🎓 Agent Vocal IA - Setup Colab

**⚠️ IMPORTANT** : Activez le GPU → Runtime → Change runtime type → T4 GPU → Save

Installation optimisée pour Python 3.12+ avec gestion automatique des erreurs.

## 📋 Étape 1 : Vérification GPU

In [ ]:
import subprocess, sys, platform
print("=" * 60)
print(f"🖥️  Python {sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}")
print("=" * 60)
try:
    subprocess.run(['nvidia-smi'], check=True, capture_output=True)
    print("✅ GPU détecté!")
except:
    print("⚠️  Aucun GPU - Activez-le: Runtime → Change runtime type → T4 GPU")
print("=" * 60)

## 📥 Étape 2 : Clonage du projet

In [ ]:
import os, shutil
if os.path.exists('agent_vocal_IA'):
    shutil.rmtree('agent_vocal_IA')
!git clone https://github.com/Romainmlt123/agent_vocal_IA.git
%cd agent_vocal_IA
print(f"✅ Projet cloné: {os.getcwd()}")

## 📦 Étape 3 : Installation (5-10 min)

**Résout automatiquement** :
- ✅ PortAudio (sounddevice)
- ✅ NumPy <2.0 (compatibilité)
- ✅ FAISS-GPU → FAISS-CPU
- ✅ llama-cpp-python CUDA
- ✅ gTTS (Piper incompatible Python 3.12+)

In [ ]:
import sys, subprocess
print("🔄 Installation...")

# 1. PortAudio
print("\n1/8 PortAudio...")
!apt-get update -qq && apt-get install -qq -y portaudio19-dev > /dev/null 2>&1

# 2. Packages base
print("2/8 Packages base...")
!pip install -q torch torchaudio faster-whisper==1.0.3 sentence-transformers==2.7.0
!pip install -q pypdf==4.2.0 langchain==0.2.11 langchain-community==0.2.10 gradio==4.36.1

# 3. NumPy
print("3/8 NumPy...")
!pip install -q 'numpy>=1.24,<2.0'

# 4. Audio
print("4/8 Audio...")
!pip install -q pyyaml soundfile sounddevice

# 5. FAISS
print("5/8 FAISS...")
if subprocess.run(['pip','install','-q','faiss-gpu'], capture_output=True).returncode != 0:
    !pip install -q faiss-cpu==1.8.0

# 6. Silero VAD
print("6/8 Silero VAD...")
!pip install -q silero-vad

# 7. llama-cpp-python
print("7/8 llama-cpp-python...")
if subprocess.run(['pip','install','-q','llama-cpp-python','--extra-index-url','https://abetlen.github.io/llama-cpp-python/whl/cu121'], capture_output=True).returncode != 0:
    !pip install -q llama-cpp-python==0.2.85

# 8. TTS
print("8/8 TTS...")
!pip install -q gTTS pyttsx3

print("\n✅ Installation terminée!")
print("📝 TTS: gTTS installé (Piper incompatible Python 3.12+)")

## ✅ Étape 4 : Vérification

In [ ]:
print("📚 Vérification...\n")
modules = ['torch','faster_whisper','sentence_transformers','faiss','langchain','llama_cpp','gradio','numpy','sounddevice','gtts']
ok = 0
for m in modules:
    try:
        __import__(m.replace('-','_'))
        print(f"✅ {m}")
        ok += 1
    except:
        print(f"❌ {m}")
print(f"\n📊 {ok}/{len(modules)} modules OK")
print("🎉 Prêt!" if ok >= len(modules)-1 else "⚠️  Réexécutez l'installation")

## 📥 Étape 5 : Téléchargement modèle LLM (10 min)

In [ ]:
import os, urllib.request
from pathlib import Path

Path("models/llm").mkdir(parents=True, exist_ok=True)

url = "https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-q4.gguf"
dest = "models/llm/phi-3-mini-4k-instruct.Q4_K_M.gguf"

if os.path.exists(dest):
    print("✅ Modèle déjà téléchargé")
else:
    print("📥 Téléchargement Phi-3 Mini (2.4 GB)...")
    def progress(b, bs, t):
        p = min(100, b*bs*100/t) if t > 0 else 0
        print(f"\r[{'█'*int(p/2.5)}{'░'*(40-int(p/2.5))}] {p:.1f}%", end='')
    urllib.request.urlretrieve(url, dest, progress)
    print("\n✅ Téléchargé!")

## 🔍 Étape 6 : Construction indices RAG

In [ ]:
print("🔍 Construction des indices RAG...\n")
!python -m src.rag_build --subject maths
!python -m src.rag_build --subject physique
!python -m src.rag_build --subject anglais
print("\n✅ Indices construits!")

## 🚀 Étape 7 : Lancement interface

In [ ]:
from ui.app import launch_ui
print("🚀 Lancement...\n")
launch_ui(share=True)
print("\n📝 Cliquez sur le lien ci-dessus")
print("⚠️  Utilisez le mode TEXTE (TTS pas disponible sur Python 3.12+)")